### Importing the relevant libraires

In [174]:
import lancedb
import pandas as pd
from sentence_transformers import SentenceTransformer

restaurant_data = pd.read_csv("data.csv")
restaurant_data = restaurant_data[restaurant_data.columns[1:]]
restaurant_data.dropna(inplace=True)
restaurant_data.drop_duplicates(inplace=True)
restaurant_data.head()

,Area,City,Restaurant,Price,Avg ratings,Total ratings,Food type,Address,Delivery time
0,Koramangala,Bangalore,Tandoor Hut,300.0,4.4,100,"Biryani,Chinese,North Indian,South Indian",5Th Block,59
1,Koramangala,Bangalore,Tunday Kababi,300.0,4.1,100,"Mughlai,Lucknowi",5Th Block,56
2,Jogupalya,Bangalore,Kim Lee,650.0,4.4,100,Chinese,Double Road,50
3,Indiranagar,Bangalore,New Punjabi Hotel,250.0,3.9,500,"North Indian,Punjabi,Tandoor,Chinese",80 Feet Road,57
4,Indiranagar,Bangalore,Nh8,350.0,4.0,50,"Rajasthani,Gujarati,North Indian,Snacks,Desser...",80 Feet Road,63


### Embedding the relevant parts of the data.

In [175]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
data_points_vectors = []

for _, row in restaurant_data.iterrows():
    filter_cols = ['Food type', 'Avg ratings', 'Address']
    data_point = "#".join(f"{col}/{row[col]}" for col in filter_cols)
    data_points_vectors.append(data_point)

# Add the new column to the DataFrame
restaurant_data["query_string"] = data_points_vectors

list_of_payloads = []

for index, row in restaurant_data.iterrows():
    encoded_vector = model.encode(row['query_string'])
    payload = {
        'Area': row['Area'],
        'City': row['City'],
        'Restaurant': row['Restaurant'],
        'Price': row['Price'],
        'Avg_ratings': row['Avg ratings'],
        'Total_ratings': row['Total ratings'],
        'Food_type': row['Food type'],
        'Address': row['Address'],
        'Delivery_time': row['Delivery time'],
        'query_string': row['query_string'],
        'vector': encoded_vector
    }

    list_of_payloads.append(payload)

### Using the LanceDB database

In [177]:
# Connect to the LanceDB instance
uri = "data"
db = lancedb.connect(uri)

lancedb_table = db.create_table("restaurant-geocoding-app", data=list_of_payloads)

In [182]:
df = lancedb_table.to_pandas()
df.head()

,Area,City,Restaurant,Price,Avg_ratings,Total_ratings,Food_type,Address,Delivery_time,query_string,vector
0,Koramangala,Bangalore,Tandoor Hut,300.0,4.4,100,"Biryani,Chinese,North Indian,South Indian",5Th Block,59,"Food type/Biryani,Chinese,North Indian,South I...","[0.1283033, 0.14721109, -0.08635097, 0.0826363..."
1,Koramangala,Bangalore,Tunday Kababi,300.0,4.1,100,"Mughlai,Lucknowi",5Th Block,56,"Food type/Mughlai,Lucknowi#Avg ratings/4.1#Add...","[-0.10582756, 0.15009406, -0.35311964, 0.12081..."
2,Jogupalya,Bangalore,Kim Lee,650.0,4.4,100,Chinese,Double Road,50,Food type/Chinese#Avg ratings/4.4#Address/Doub...,"[-0.09362266, 0.16319275, 0.12415708, 0.012913..."
3,Indiranagar,Bangalore,New Punjabi Hotel,250.0,3.9,500,"North Indian,Punjabi,Tandoor,Chinese",80 Feet Road,57,"Food type/North Indian,Punjabi,Tandoor,Chinese...","[0.1270527, 0.17128171, 0.013175009, 0.2396797..."
4,Indiranagar,Bangalore,Nh8,350.0,4.0,50,"Rajasthani,Gujarati,North Indian,Snacks,Desser...",80 Feet Road,63,"Food type/Rajasthani,Gujarati,North Indian,Sna...","[0.0823844, 0.014472799, -0.11513429, 0.284303..."


In [179]:
df["query_string"][0]

'Food type/Biryani,Chinese,North Indian,South Indian#Avg ratings/4.4#Address/5Th Block'

### Extracting the specifics from the query 

In [187]:
query_string = "Hi, I am looking for a casual dining restaurant where Indian or Italian food is served near the HSR Bangalore"

# Helper prompt to extract structured data from ip_prompt
total_prompt = f"""Query String: {query_string}\n\n\
Now from the query string above extract these following entities pinpoints:
1. Food type : Extract the food type 
2. Avg ratings : Extract the average ratings
3. Address : Extract the current exact location, don't consider the fillers like "near" or "nearby".

NOTE : For the Current location, try to understand the pin point location in the query string. Do not give any extra information. If you make the mistakes, bad things
will happen.

Finally return a python dictionary using those points as keys and don't write the markdown of python. If value of a key is not mentioned, then set it as None.
"""

# Make a request to OpenAI's API
completion = client.chat.completions.create(
    model="gpt-4o",  # Use the appropriate model
    store=True,
    messages=[
        {"role": "user", "content": total_prompt}
    ]
)

# Extract the generated text
content = completion.choices[0].message.content
print(content)

{
    "Food type": "Indian or Italian",
    "Avg ratings": None,
    "Address": "HSR Bangalore"
}


### Addding the response dictionary to the response.

In [185]:
import ast

# Convert the string content to a dictionary
try:
    response_dict = ast.literal_eval(content)
except (ValueError, SyntaxError) as e:
    print("Error parsing the response:", e)
    response_dict = {}


filter_cols = ['Food type', 'Avg ratings', 'Address']
query_string_parts = [f"{col}/{response_dict.get(col)}" for col in filter_cols if response_dict.get(col)]

query_string = "#".join(query_string_parts)
print((query_string))

Food type/Indian or Italian#Address/HSR Bangalore


### Using LanceDB FTS for searching

In [190]:
# Create the FTS index and search
lancedb_table.create_fts_index("query_string", replace=True)
results = lancedb_table.search(query_string).to_pandas()

### GeoSpatial Recommendation

In [166]:
import requests
import math

def get_google_geocoding(address, api_key):
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": address, "key": api_key}
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        result = response.json()
        if result["status"] == "OK":
            latitude = result["results"][0]["geometry"]["location"]["lat"]
            longitude = result["results"][0]["geometry"]["location"]["lng"]
            return (latitude, longitude)
        else:
            print(f"Google API: No results found for address: {address}")
            return None
    else:
        print(f"Google API: Request failed for address: {address}")
        return None

def haversine(coord1, coord2):
    R = 6371.0  # Radius of the Earth in kilometers
    lat1, lon1 = map(math.radians, coord1)
    lat2, lon2 = map(math.radians, coord2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

def process_top_restaurants(data, current_location, api_key, top_n=5):
    current_coords = get_google_geocoding(current_location, api_key)
    if not current_coords:
        return

    for index, row in data.head(top_n).iterrows():
        complete_address = f"{row['Restaurant']}, {row['City']}"
        restaurant_coords = get_google_geocoding(complete_address, api_key)
        if restaurant_coords:
            distance = haversine(current_coords, restaurant_coords)
            print(f"Restaurant Name: {row['Restaurant']}")
            print(f"Distance: {distance:.2f} km")
            print(f"Area: {row['Area']}")
            print(f"Price: {row['Price']}")
            print(f"Coordinates: {restaurant_coords}")
            print(f"Cuisines Type: {row['Food_type']}")
            print("-" * 40)

# Example usage
api_key = '*****'
current_location = 
process_top_restaurants(results, current_location, api_key, top_n=5)

Restaurant Name: Brooks And Bonds Brewery
Distance: 3.36 km
Area: Koramangala
Price: 200.0
Coordinates: (12.9341801, 77.62334249999999)
Cuisines Type: Indian
----------------------------------------
Restaurant Name: Cafe Azzure
Distance: 8.06 km
Area: Ashok Nagar
Price: 1000.0
Coordinates: (12.975012, 77.6076558)
Cuisines Type: American,Italian
----------------------------------------
Restaurant Name: Tottos Pizza
Distance: 7.92 km
Area: Central Bangalore
Price: 500.0
Coordinates: (12.9731935, 77.607012)
Cuisines Type: Continental,Italian
----------------------------------------
Restaurant Name: Holy Doh!
Distance: 4.15 km
Area: Central Bangalore
Price: 600.0
Coordinates: (12.9346188, 77.6139914)
Cuisines Type: Pizzas,Italian
----------------------------------------
Restaurant Name: Bakery By Foodhall
Distance: 7.31 km
Area: Ulsoor
Price: 300.0
Coordinates: (12.9734944, 77.62038629999999)
Cuisines Type: Bakery,Italian,Desserts
----------------------------------------
